```@schema
class ScanInfo(dj.Imported):
    definition = """ # general data about the reso/meso scans from header
    -> Scan
    ---
    nfields              : tinyint   # number of fields
    nchannels            : tinyint   # number of channels
    ndepths              : int       # Number of scanning depths (planes)
    nframes              : int       # number of recorded frames
    nrois                : tinyint   # number of ROIs (see scanimage's multi ROI imaging)
    x=null               : float     # (um) ScanImage's 0 point in the motor coordinate system
    y=null               : float     # (um) ScanImage's 0 point in the motor coordinate system
    z=null               : float     # (um) ScanImage's 0 point in the motor coordinate system
    fps                  : float     # (Hz) frames per second - Volumetric Scan Rate 
    bidirectional        : boolean   # true = bidirectional scanning
    usecs_per_line=null  : float     # microseconds per scan line
    fill_fraction=null   : float     # raster scan temporal fill fraction (see scanimage)
    scan_datetime=null   : datetime  # datetime of the scan
    scan_duration=null   : float     # (seconds) duration of the scan
    """

    class Field(dj.Part):
        definition = """ # field-specific scan information
        -> master
        field_idx         : int
        ---
        px_height         : smallint  # height in pixels
        px_width          : smallint  # width in pixels
        um_height=null    : float     # height in microns
        um_width=null     : float     # width in microns
        field_x=null      : float     # (um) center of field in the motor coordinate system
        field_y=null      : float     # (um) center of field in the motor coordinate system
        field_z=null      : float     # (um) relative depth of field
        delay_image=null  : longblob  # (ms) delay between the start of the scan and pixels in this field
        roi=null          : int       # the scanning roi (as recorded in the acquisition software) containing this field - only relevant to mesoscale scans
        """
        ```

In [36]:
import xml.etree.ElementTree as ET
import numpy as np
from datetime import datetime

In [37]:
tree = ET.parse("../user_data/A_1040736_Ses_1_Planes_3_LP241_Gain_740-001.xml")
root = tree.getroot()

In [38]:
nfields = 1      # Resonance scanner

In [4]:
channel_list = list()
channels = root.iterfind(".//Sequence/Frame/File/[@channel]")
for channel in channels:
    channel_list.append(int(channel.attrib.get('channel')))
nchannels = np.unique(channel_list).shape[0]

In [41]:
root.findall(".//Sequence/[@cycle='1']/Frame")

[<Element 'Frame' at 0x0000017ACB9CA180>,
 <Element 'Frame' at 0x0000017ACB9CA4F0>,
 <Element 'Frame' at 0x0000017ACB9CA720>]

In [42]:
planes_list = []
planes = root.findall(".//Sequence/[@cycle='1']/Frame")
for plane in planes:
    planes_list.append(int(plane.attrib.get('index')))

ndepths = np.unique(planes_list).shape[0]

In [5]:
planes_list = list()
planes = root.iterfind(".//Sequence/Frame/[@index]")
for plane in planes:
    planes_list.append(int(plane.attrib.get('index')))

ndepths = np.unique(planes_list).shape[0]

In [6]:
n_frames = root.findall('.//Sequence')[-1].attrib.get('cycle')

In [7]:
roi = 1     # Resonance scanner

In [8]:
x_coordinate = root.find(".//PVStateValue/[@key='currentScanCenter']/IndexedValue/[@index='XAxis']").attrib.get('value')

y_coordinate = root.find(".//PVStateValue/[@key='currentScanCenter']/IndexedValue/[@index='YAxis']").attrib.get('value')

z_coordinate = root.find(".//PVStateValue/[@key='positionCurrent']/SubindexedValues/[@index='ZAxis']/SubindexedValue/[@subindex='0']").attrib.get('value')

In [13]:
z_coordinate

'200'

In [9]:
frame_rate = np.divide(1, float(root.findall('.//PVStateValue/[@key="framePeriod"]')[0].attrib.get('value')))   # rate = 1/framePeriod

In [10]:
scan_datetime = datetime.strptime(root.attrib.get('date'), "%m/%d/%Y %I:%M:%S %p")

In [11]:
usec_per_line = float(root.findall(".//PVStateValue/[@key='scanLinePeriod']")[0].attrib.get('value')) * 1000    # multiply to convert to microseconds

In [29]:
px_height = root.findall(".//PVStateValue/[@key='pixelsPerLine']")[0].attrib.get('value')
px_width = px_height    # All PrairieView-acquired images have square dimensions (512 x 512; 1024 x 1024)

In [17]:
root.findall(".//Sequence/[@cycle='1']/Frame")

[<Element 'Frame' at 0x0000017AA2D2C180>,
 <Element 'Frame' at 0x0000017AA2D2C4F0>,
 <Element 'Frame' at 0x0000017AA2D2C720>]

In [14]:
bidirectionalZ = bool(root.find(".//Sequence").attrib.get('bidirectionalZ'))

Make a list through ndepths. Reverse it, and pass it as the new index value

idx_range = list(range(1, ndepths+1))
rev_idx = idx_range[::-1]

ndepths = 3
idx_range = list(range(1, ndepths+1))
rev_idx = idx_range[::-1]
for sequence in root.findall(".//Sequence/[@cycle]"):
    if int(sequence.attrib.get('cycle')) % 2 == 0:
        frames = sequence.findall(".//Frame")
        

[(count, frame) for count, frame in enumerate(sequence.findall(".//Frame"))]

In [ ]:
for node in root.findall(".//Sequence/[@cycle]"):
    if int(node.attrib.get('cycle')) % 2 == 0:
        tags = node.findall("./Frame")
        for tag in tags:
            try:
                idx = int(tag.attrib.get('index'))
                if idx == 3:
                    tag.attrib['index'] = '1'
                elif idx == 1:
                    tag.attrib['index'] = '3'
                else:
                    pass
            except TypeError:
                pass

In [ ]:
sequences = root.findall(".//Sequence")
for seq in sequences:
    cycle = int(seq.attrib.get('cycle'))
    max_cycle_idx = cycle * ndepths
    frames = seq.findall(".//Frame")
    for idx in range(0, ndepths):
        index = seq.findall(".//Frame")[idx].attrib.get('index')
        field_idx = max_cycle_idx - (ndepths-int(index))
        frames[idx].set('field_idx', str(field_idx))       


In [19]:
um_per_pixel = float(root.find(".//PVStateValue/[@key='micronsPerPixel']/IndexedValue/[@index='XAxis']").attrib.get('value'))
um_height = float(root.findall(".//PVStateValue/[@key='pixelsPerLine']")[0].attrib.get('value')) * um_per_pixel
um_width = um_height    # All PrairieView-acquired images have square dimensions (512 x 512; 1024 x 1024)

In [23]:
total_duration = root.findall(".//Sequence/Frame")[-1].attrib.get('relativeTime')

In [24]:
total_duration

'932.081788005'

In [20]:
um_height

742.0

In [30]:
meta = dict(
        num_fields = nfields,
        num_channels = nchannels,
        num_planes = ndepths,
        num_frames = n_frames,
        num_rois = roi,
        x_pos = x_coordinate,
        y_pos = y_coordinate,
        z_pos = z_coordinate,
        bidirectional_z = bidirectionalZ,
        scan_datetime = scan_datetime,
        scan_duration = total_duration, 
        height_in_pixels = px_height,
        width_in_pixels = px_width,
        height_in_um = um_height,
        width_in_um = um_width,
    )

In [35]:
meta["num_fields"]

1